In [195]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk
# % matplotlib inline
#plt.rcParams['animation.html'] = 'jshtml'
from IPython.display import HTML

from copy import deepcopy
import os
import sys
from functools import reduce
sys.path.append("/home/ful7rng/projects/transition/propagation")
from utils import pickle_load, get_npy, get_file
from utils.occ_map_utils import load_map, show_map, plot_occ_map, plot_trajectories, free_space
from utils.plot_utils import plot_4d_tensor
from utils.scene_utils import display_scenes_trajs, animate_scenes, get_scenes
from propagation.bofum import conditionalBOFUM, naiveBOFUM, BOFUMRealdata
from propagation.animation import TrackingAnimRealdata, Plot
from data_loader import get_map_crop


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
plt.ion()

In [47]:
diagonal = True
if diagonal:
    model_name = '04_DIAGONAL_TRUE_CONDITIONAL_TRUE'
else:
    model_name = '20_ALL_MAPS_W_MASK'
    # model_name = '01_JOINT_PROB_ALL_MAPS'

### show nn_output and kernel at map location

In [48]:
from propagation.bofum import conditionalBOFUM, naiveBOFUM
from data_loader import get_map_crop

config_path = '/local/home/ful7rng/projects/transition/config_diagonal_True_conditional_True.py'
maps, outputs = get_map_crop(config_path, num=1, dataset='train', probs=True)
map_ = maps[0]
output = outputs[0]

(27119, 32, 32)


In [49]:
output.shape

(32, 32, 8, 8)

In [61]:
x, y = np.random.randint(0, 32), np.random.randint(0, 32)
print([x, y])
print(output[x, y])
print(np.sum(output[x, y], axis=1))

[31, 19]
[[ 0.          0.          0.          0.          0.          0.          1.
   0.        ]
 [ 0.          0.          0.          0.          0.          0.          1.
   0.        ]
 [ 0.          0.          0.          0.          0.          0.          1.
   0.        ]
 [ 0.          0.          0.          0.          0.          0.          1.
   0.        ]
 [ 0.          0.          0.          0.          0.          0.          1.
   0.        ]
 [ 0.08474576  0.08474576  0.08474576  0.08474576  0.08474576  0.49152542
   0.          0.08474576]
 [ 0.          0.          0.          0.          0.          0.          1.
   0.        ]
 [ 0.          0.          0.          0.          0.          0.          1.
   0.        ]]
[ 1.  1.  1.  1.  1.  1.  1.  1.]


In [62]:
kwargs = dict(omega=0.05,
              noise_var=0.5,
              extent=7,
              verbose=False)

bofum = conditionalBOFUM(map_, model_name,
                                   name = 'BOFUM with n. o. as acceleration',
                                   simulated_data=True,
                                   nn_probs=output,
                                   force_predict=True,
                                   **kwargs)

In [63]:
distances, trajs = bofum.initialize(num_targets=3, num_steps=20, diagonal=diagonal)

sampling trajectories: 1/3


In [64]:
plt.figure()
bofum.show_trajectories()

In [65]:


def onclick(event, bofum, fig, fig_1, fig_2, plot):
    ix, iy = event.xdata, event.ydata
    coords = np.floor(np.array([ix, iy]) / bofum.map_res).astype(int)
    print(coords)
    
    clicked = np.zeros_like(bofum.map)
    x, y = coords[0], coords[1]
    clicked[x, y] = 1
    plot.set_axes_data("occupancy_axes", clicked)
    fig.canvas.draw()
    
    fig_1.clear()
    fig_2.clear()
    probs = bofum.nn_probs
    prob = probs[x, y]
    print(np.rot90(np.sum(prob, axis=(2, 3))))
    print(np.sum(prob))
    plot_4d_tensor(prob, fig=fig_1)
    plot_4d_tensor(bofum.kernels[x, y], fig=fig_2)
    print("kernel sum is : "+ str(np.sum(bofum.kernels[x, y])))
    #plot_4d_tensor(transition_counts[x, y], fig=fig_2)
    fig_1.canvas.draw()
    fig_2.canvas.draw()

In [66]:
from propagation.animation import Plot
fig = plt.figure(figsize=(5, 5))
fig_1 = plt.figure(figsize=(6, 5))
fig_2 = plt.figure(figsize=(6, 5))

map_axes = fig.add_subplot(111)
plot = Plot(map_axes, bofum.map, bofum.map_res)
fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, bofum, fig, fig_1, fig_2, plot))

7

[10 19]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[23 14]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[ 7 18]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[ 7 16]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[ 1 14]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[ 2 14]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[ 2 14]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[ 1 14]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[ 2 14]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[ 2 13]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[ 2 12]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[ 8 14]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]
8.0
kernel sum is : 49.0
[ 9 14]
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]

In [46]:
kwargs = dict(omega=0.05,
              noise_var=0.5,
              extent=7,
              verbose=False)

bofum_1 = conditionalBOFUM(map_, model_name,
                                   name = 'BOFUM with n. o. as acceleration',
                                   simulated_data=True,
                                   with_reachability=True,
                                   acceleration_interpretation=True,
                                   nn_probs=output,
                                   force_predict=True,
                                   conditional=conditional,
                                   config=config,
                                   **kwargs)

distances, trajs = bofum_1.initialize(num_targets=1, num_steps=20, diagonal=False)

from propagation.animation import Plot
fig_3 = plt.figure(figsize=(5, 5))
fig_4 = plt.figure(figsize=(6, 5))
fig_5 = plt.figure(figsize=(6, 5))

map_axes_1 = fig_3.add_subplot(111)
plot_1 = Plot(map_axes_1, bofum_1.map, bofum_1.map_res)
fig_3.canvas.mpl_connect('button_press_event', lambda event: onclick(event, bofum_1, fig_3, fig_4, fig_5, plot_1))

NameError: name 'conditional' is not defined

[12 17]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0
[10 17]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0
[12 15]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0
[12 16]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0
[8 6]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0
[8 5]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0
[8 4]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0
[18  8]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0
[15  5]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0
[15  4]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0
[15  5]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0
[15  4]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
kernel sum is : 49.0


In [34]:
kwargs = dict(omega=0.05,
              noise_var=0.5,
              extent=7,
              verbose=False)

bofum_2 = conditionalBOFUM(map_, model_name,
                                   name = 'BOFUM with n. o. as acceleration',
                                   simulated_data=True,
                                   with_reachability=True,
                                   acceleration_interpretation=True,
                                   nn_probs=None,
                                   force_predict=True,
                                   conditional=conditional,
                                   config=config,
                                   **kwargs)

distances, trajs = bofum_2.initialize(num_targets=1, num_steps=20, diagonal=False)

from propagation.animation import Plot
fig_6 = plt.figure(figsize=(5, 5))
fig_7 = plt.figure(figsize=(6, 5))
fig_8 = plt.figure(figsize=(6, 5))

map_axes_2 = fig_6.add_subplot(111)
plot_2 = Plot(map_axes_2, bofum_2.map, bofum_2.map_res)
fig_6.canvas.mpl_connect('button_press_event', lambda event: onclick(event, bofum_2, fig_6, fig_7, fig_8, plot_2))

---------------------------------------------------------------------------
---------------------------------------------------------------------------

    Layer           Output shape         W shape             
1   Input           (None, 1, None, None)                     
2   Conv2D          (None, 8, None, None) (8, 1, 1, 1)        
3   Conv2D          (None, 12, None, None) (12, 8, 3, 3)       
4   Concat          (None, 20, None, None)                     
5   Conv2D          (None, 12, None, None) (12, 20, 3, 3)      
6   Concat          (None, 32, None, None)                     
7   Conv2D          (None, 12, None, None) (12, 32, 3, 3)      
8   Concat          (None, 44, None, None)                     
9   Conv2D          (None, 12, None, None) (12, 44, 3, 3)      
10  Concat          (None, 56, None, None)                     
11  Conv2D          (None, 12, None, None) (12, 56, 3, 3)      
12  Concat          (None, 68, None, None)                     
13  Conv2D         

7

[28 26]
[[ 0.          0.16129032  0.        ]
 [ 0.37903226  0.          0.2983871 ]
 [ 0.          0.16129032  0.        ]]
1.0
kernel sum is : 49.0
[27 26]
[[ 0.          0.31111111  0.        ]
 [ 0.20555556  0.          0.16111111]
 [ 0.          0.32222222  0.        ]]
1.0
kernel sum is : 49.0
[27 26]
[[ 0.          0.4575373   0.        ]
 [ 0.0235649   0.          0.03274885]
 [ 0.          0.48614893  0.        ]]
0.999999977183
kernel sum is : 49.0
[28 26]
[[ 0.          0.39228304  0.        ]
 [ 0.09305708  0.          0.08540007]
 [ 0.          0.4292598   0.        ]]
0.999999990687
kernel sum is : 49.0
[29 26]
[[ 0.          0.33182057  0.        ]
 [ 0.1566642   0.          0.14354769]
 [ 0.          0.36796752  0.        ]]
0.999999977648
kernel sum is : 49.0
[29 26]
[[ 0.        0.309375  0.      ]
 [ 0.14375   0.        0.175   ]
 [ 0.        0.371875  0.      ]]
1.0
kernel sum is : 49.0


### validate rescale trajectory and recover

In [27]:
traj = trajs[0]
# rescale traj
res = 0.2
traj_scaled = np.array(traj + np.array([0.5] * 2)) * res
# recover traj
traj_recoverd = np.round((traj_scaled / res) - 0.5).astype(int)

assert (traj_recoverd == traj).all()

### validate transition probs

In [225]:
from data_generator.human_mcm import Grid_HMM
from data_generator.astar_ped_sim.astar_traj_generator import sample_trajectories

res = .2
trajs = sample_trajectories(map_, 5, mode='transverse', verbose=True, diagonal=False)

sampling trajectories for possible start locations: 1/460
sampling trajectories for possible start locations: 2/460
sampling trajectories for possible start locations: 3/460
sampling trajectories for possible start locations: 4/460
sampling trajectories for possible start locations: 5/460
sampling trajectories for possible start locations: 6/460
sampling trajectories for possible start locations: 7/460
sampling trajectories for possible start locations: 8/460
sampling trajectories for possible start locations: 9/460
sampling trajectories for possible start locations: 10/460
sampling trajectories for possible start locations: 11/460
sampling trajectories for possible start locations: 12/460
sampling trajectories for possible start locations: 13/460
sampling trajectories for possible start locations: 14/460
sampling trajectories for possible start locations: 15/460
sampling trajectories for possible start locations: 16/460
sampling trajectories for possible start locations: 17/460
sampli

sampling trajectories for possible start locations: 140/460
sampling trajectories for possible start locations: 141/460
sampling trajectories for possible start locations: 142/460
sampling trajectories for possible start locations: 143/460
sampling trajectories for possible start locations: 144/460
sampling trajectories for possible start locations: 145/460
sampling trajectories for possible start locations: 146/460
sampling trajectories for possible start locations: 147/460
sampling trajectories for possible start locations: 148/460
sampling trajectories for possible start locations: 149/460
sampling trajectories for possible start locations: 150/460
sampling trajectories for possible start locations: 151/460
sampling trajectories for possible start locations: 152/460
sampling trajectories for possible start locations: 153/460
sampling trajectories for possible start locations: 154/460
sampling trajectories for possible start locations: 155/460
sampling trajectories for possible start

sampling trajectories for possible start locations: 280/460
sampling trajectories for possible start locations: 281/460
sampling trajectories for possible start locations: 282/460
sampling trajectories for possible start locations: 283/460
sampling trajectories for possible start locations: 284/460
sampling trajectories for possible start locations: 285/460
sampling trajectories for possible start locations: 286/460
sampling trajectories for possible start locations: 287/460
sampling trajectories for possible start locations: 288/460
sampling trajectories for possible start locations: 289/460
sampling trajectories for possible start locations: 290/460
sampling trajectories for possible start locations: 291/460
sampling trajectories for possible start locations: 292/460
sampling trajectories for possible start locations: 293/460
sampling trajectories for possible start locations: 294/460
sampling trajectories for possible start locations: 295/460
sampling trajectories for possible start

sampling trajectories for possible start locations: 429/460
sampling trajectories for possible start locations: 430/460
sampling trajectories for possible start locations: 431/460
sampling trajectories for possible start locations: 432/460
sampling trajectories for possible start locations: 433/460
sampling trajectories for possible start locations: 434/460
sampling trajectories for possible start locations: 435/460
sampling trajectories for possible start locations: 436/460
sampling trajectories for possible start locations: 437/460
sampling trajectories for possible start locations: 438/460
sampling trajectories for possible start locations: 439/460
sampling trajectories for possible start locations: 440/460
sampling trajectories for possible start locations: 441/460
sampling trajectories for possible start locations: 442/460
sampling trajectories for possible start locations: 443/460
sampling trajectories for possible start locations: 444/460
sampling trajectories for possible start

In [74]:
len(trajs)

2500

In [48]:
trajs[0].shape

(25, 2)

In [66]:
from utils.occ_map_utils import plot_trajectories
fig = plt.figure()
for traj in trajs:
    plot_trajectories([traj], map_, .2)

In [13]:
len(trajs)

2170

In [17]:
trajs[0][:5]

array([[ 0, 11],
       [ 0, 12],
       [ 0, 13],
       [ 1, 13],
       [ 2, 13]])

In [226]:
diagonal = False
conditional = False
mcm = Grid_HMM(np.array(map_.shape).astype(int))
for idx, trajectory in enumerate(trajs):
    if idx % 100 == 0:
        print("Add transitions of {}/{} trajectory to the mcm.".format(idx+1, len(trajs)))
    #trajectory = np.round((trajectory / res) - 0.5).astype(int)
    #print(str(trajectory))
    for t in range(trajectory.shape[0] - 2):
        # give first cell seven prior locations
        # and add them to transitions
        if t == 0:
            first_cell = trajectory[0, :]
            second_cell = trajectory[1, :]
            velocities =  [[0, 1], [1, 0], [0, -1], [-1, 0],
                      [1, 1], [1, -1], [-1, 1], [-1, -1]]
            if not diagonal:
                velocities = velocities[:4]
            prior_cells = [[first_cell[0]+i, first_cell[1]+j]
                           for i, j in velocities]
            for cell in prior_cells:
                if cell[0] != second_cell[0] or cell[1] != second_cell[1]:
                    mcm.add_transition(cell, first_cell, second_cell)

        from_ = trajectory[t, :]
        current = trajectory[t + 1, :]
        to = trajectory[t + 2, :]
        mcm.add_transition(from_, current, to)

transition_probs = mcm.get_transition_probs(conditional)

Add transitions of 1/2300 trajectory to the mcm.
Add transitions of 101/2300 trajectory to the mcm.
Add transitions of 201/2300 trajectory to the mcm.
Add transitions of 301/2300 trajectory to the mcm.
Add transitions of 401/2300 trajectory to the mcm.
Add transitions of 501/2300 trajectory to the mcm.
Add transitions of 601/2300 trajectory to the mcm.
Add transitions of 701/2300 trajectory to the mcm.
Add transitions of 801/2300 trajectory to the mcm.
Add transitions of 901/2300 trajectory to the mcm.
Add transitions of 1001/2300 trajectory to the mcm.
Add transitions of 1101/2300 trajectory to the mcm.
Add transitions of 1201/2300 trajectory to the mcm.
Add transitions of 1301/2300 trajectory to the mcm.
Add transitions of 1401/2300 trajectory to the mcm.
Add transitions of 1501/2300 trajectory to the mcm.
Add transitions of 1601/2300 trajectory to the mcm.
Add transitions of 1701/2300 trajectory to the mcm.
Add transitions of 1801/2300 trajectory to the mcm.
Add transitions of 1901/

In [228]:
transition_counts = mcm.get_transition_counts()

[20  4]
[[ 0.          0.20547945  0.        ]
 [ 0.26712329  0.          0.26027397]
 [ 0.          0.26712329  0.        ]]
1.0
[20  1]
[[ 0.          0.16666667  0.        ]
 [ 0.16666667  0.          0.16666667]
 [ 0.          0.5         0.        ]]
1.0
[20  2]
[[ 0.          0.2826087   0.        ]
 [ 0.10869565  0.          0.10869565]
 [ 0.          0.5         0.        ]]
1.0
[20  3]
[[ 0.          0.14935065  0.        ]
 [ 0.32467532  0.          0.32467532]
 [ 0.          0.2012987   0.        ]]
1.0
[20  4]
[[ 0.          0.20547945  0.        ]
 [ 0.26712329  0.          0.26027397]
 [ 0.          0.26712329  0.        ]]
1.0
[20  5]
[[ 0.          0.0376569   0.        ]
 [ 0.46025105  0.          0.48849372]
 [ 0.          0.01359833  0.        ]]
1.0


In [223]:
x, y = np.random.randint(0, 32), np.random.randint(0, 32)
print(x, y)
print(transition_counts[x, y])
print(np.sum(transition_counts[x, y], axis=(2, 3)))

(26, 8)
[[[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  1.  0.]
   [ 1.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]


 [[[ 0.  1.  0.]
   [ 1.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  1.  0.]
   [ 0.  0.  1.]
   [ 0.  1.  0.]]]


 [[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 1.  0.  1.]
   [ 0.  1.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]]
[[ 0.  2.  0.]
 [ 2.  0.  3.]
 [ 0.  3.  0.]]


In [93]:
x, y = np.random.randint(0, 32), np.random.randint(0, 32)
print(x, y)
print(transition_probs[x, y])
print(np.sum(transition_probs[x, y], axis=(2, 3)))

(26, 21)
[[[[ 0.   0.   0. ]
   [ 0.   0.   0. ]
   [ 0.   0.   0. ]]

  [[ 0.   0.1  0. ]
   [ 0.   0.   0.1]
   [ 0.   0.   0. ]]

  [[ 0.   0.   0. ]
   [ 0.   0.   0. ]
   [ 0.   0.   0. ]]]


 [[[ 0.   0.1  0. ]
   [ 0.1  0.   0. ]
   [ 0.   0.1  0. ]]

  [[ 0.   0.   0. ]
   [ 0.   0.   0. ]
   [ 0.   0.   0. ]]

  [[ 0.   0.1  0. ]
   [ 0.   0.   0.1]
   [ 0.   0.   0. ]]]


 [[[ 0.   0.   0. ]
   [ 0.   0.   0. ]
   [ 0.   0.   0. ]]

  [[ 0.   0.   0. ]
   [ 0.1  0.   0.1]
   [ 0.   0.1  0. ]]

  [[ 0.   0.   0. ]
   [ 0.   0.   0. ]
   [ 0.   0.   0. ]]]]
[[ 0.   0.2  0. ]
 [ 0.3  0.   0.2]
 [ 0.   0.3  0. ]]


In [227]:
kwargs = dict(omega=0.05,
              noise_var=0.5,
              extent=7,
              verbose=False)

#model_name = '23_ALL_MAPS_EIGHT_DIRECTIONS'
model_name = '20_ALL_MAPS_W_MASK'

bofum_2 = conditionalBOFUM(map_, model_name,
                                   name = 'BOFUM with n. o. as acceleration',
                                   simulated_data=True,
                                   with_reachability=True,
                                   acceleration_interpretation=True,
                                   nn_probs=None,
                                   force_predict=True,
                                   conditional=conditional,
                                   config=config,
                                   **kwargs)

bofum_2.nn_probs = transition_probs
distances, trajs = bofum_2.initialize(num_targets=1, num_steps=20, diagonal=False)

from propagation.animation import Plot
fig_6 = plt.figure(figsize=(5, 5))
fig_7 = plt.figure(figsize=(6, 5))
fig_8 = plt.figure(figsize=(6, 5))

map_axes_2 = fig_6.add_subplot(111)
plot_2 = Plot(map_axes_2, bofum_2.map, bofum_2.map_res)
fig_6.canvas.mpl_connect('button_press_event', lambda event: onclick(event, bofum_2, fig_6, fig_7, fig_8, plot_2))

sampling trajectories: 1/1


7

[20  5]
[[ 0.          0.0376569   0.        ]
 [ 0.46025105  0.          0.48849372]
 [ 0.          0.01359833  0.        ]]
1.0


### validate transition_maps

In [46]:
from utils.occ_map_utils import load_map, show_map, plot_occ_map, plot_trajectories, free_space

simple_map, _, _ = load_map("/home/ful7rng/projects/transition/data_validation/maps/simple_map/thresholded_20.png")
tran_map = np.load("/home/ful7rng/projects/transition/data_validation/transition_maps/simple_map/astar_cost_transverse_6_20_5/astar_cost_transverse_6_20_5.npy")

In [ ]:
directions = {0: 'UP', 1: 'RIGHT', 2: 'DOWN', 3: 'LEFT'}

In [47]:
m = 0
n = 0
fig = plt.figure()
show_map(simple_map, resolution=0.2)
plot_occ_map(tran_map[:,:,m,n], simple_map, occ_map_res=0.2)
fig.suptitle('P[{}|{}]'.format(directions[m], directions[n]))
plt.show()

NameError: name 'probs' is not defined

In [67]:
np.ravel_multi_index((2, 1), (3, 2))

5

### validate data agumentation

In [13]:

def show_map_and_probs(map_, transition_probs_1):

    kwargs = dict(omega=0.05,
                  noise_var=0.5,
                  extent=7,
                  verbose=False)

    #model_name = '23_ALL_MAPS_EIGHT_DIRECTIONS'
    model_name = '20_ALL_MAPS_W_MASK'

    bofum_2 = conditionalBOFUM(map_, model_name,
                                       name = 'BOFUM with n. o. as acceleration',
                                       simulated_data=True,
                                       with_reachability=True,
                                       acceleration_interpretation=True,
                                       nn_probs=None,
                                       force_predict=True,
                                       conditional=conditional,
                                       config=config,
                                       **kwargs)

    bofum_2.conditional_probs = transition_probs_1
    distances, trajs = bofum_2.initialize(num_targets=1, num_steps=20, diagonal=False)

    from propagation.animation import Plot
    fig_6 = plt.figure(figsize=(5, 5))
    fig_7 = plt.figure(figsize=(6, 5))
    fig_8 = plt.figure(figsize=(6, 5))

    map_axes_2 = fig_6.add_subplot(111)
    plot_2 = Plot(map_axes_2, bofum_2.map, bofum_2.map_res)
    fig_6.canvas.mpl_connect('button_press_event', lambda event: onclick(event, bofum_2, fig_6, fig_7, fig_8, plot_2))

#### flip left/right

In [15]:
def flip(probs):
    # spatial dimensions of numpy array flip updown
    probs = np.flip(probs, axis=0)
    # velocity directions of left/right change to right/left
    probs = np.flip(probs, axis=2)
    probs = np.flip(probs, axis=4)
    return probs

In [16]:
map_1 = np.flipud(map_.copy())
trans_probs_1 = flip(transition_probs.copy())

In [17]:
show_map_and_probs(map_1, trans_probs_1)

sampling trajectories: 1/1
[25 14]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[14 10]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[15  9]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[16  9]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[17  9]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[17 10]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[25 16]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[ 6 16]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[ 6 16]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[12 11]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[12  8]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[19  8]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0


#### rotate 90

In [20]:
def rotate(probs, k):
    """ Rotate the probs 90 degrees counter-clockwise k times.

    Note that directions of probabilities change after rotations. For example, if rotate once,
    the probability of P{left|down} changes to P{down|right}.
    """
    if k == 0:
        return probs

    # spatial dimensions rotate counter-clockwise
    probs = np.rot90(probs, k)
    # velocitiy dimensions rotate clockwise
    probs = np.rot90(probs, k, (2, 3))
    probs = np.rot90(probs, k, (4, 5))
    return probs

In [21]:
k = 1

In [26]:
map_2 = np.rot90(map_.copy(), k)
trans_probs_2 = rotate(transition_probs.copy(), k)

In [27]:
show_map_and_probs(map_2, trans_probs_2)

sampling trajectories: 1/1
[ 7 14]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[17  7]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[14  9]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[24 14]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[21 14]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[21 15]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[22 15]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0
[22 14]
[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
4.0


In [67]:
?np.expand_dims

### check number of probs from nn output

In [231]:
diagonal = False
if diagonal:
    velocities = [[0, 1], [1, 0], [0, -1], [-1, 0],
                  [1, 1], [1, -1], [-1, 1], [-1, -1]]
else:
    velocities = [[0, 1], [1, 0], [0, -1], [-1, 0]]

In [232]:
def get_backward_vel(v1, v2):
    backward_v1 = [-v2[0], -v2[1]]
    backward_v2 = [-v1[0], -v1[1]]
    return backward_v1, backward_v2

In [233]:

vs = []
idxs = []
for v1 in velocities:
    for v2 in velocities:
        idx1 = Grid_HMM.two_d_vel_to_idx(v1)
        idx2 = Grid_HMM.two_d_vel_to_idx(v2)
        if v1 == [-v2[0], -v2[1]]:
            print("exclude :" + str(v1))
            continue
        backward_v1, backward_v2 = get_backward_vel(v1, v2) 
        b_idx1 = Grid_HMM.two_d_vel_to_idx(backward_v1)
        b_idx2 = Grid_HMM.two_d_vel_to_idx(backward_v2)
        v1_, v2_, backward_v1_, backward_v2_ = map(lambda foo: tuple(foo), [v1, v2, backward_v1, backward_v2])
        idx1_, idx2_, b_idx1_, b_idx2_ = map(lambda foo: tuple(foo), [idx1, idx2, b_idx1, b_idx2])
        if (v1_ + v2_) not in vs and (backward_v1_ + backward_v2_) not in vs:
            vs.append((v1_ + v2_))
        if (idx1_ + idx2_) not in idxs and (b_idx1_ + b_idx2_) not in idxs:
            idxs.append((idx1_ + idx2_))
            

exclude :[0, 1]
exclude :[1, 0]
exclude :[0, -1]
exclude :[-1, 0]


In [187]:
vs

[(0, 1, 0, 1),
 (0, 1, 1, 0),
 (0, 1, -1, 0),
 (1, 0, 0, 1),
 (1, 0, 1, 0),
 (0, -1, 1, 0)]

In [234]:
idxs

[(1, 2, 1, 2),
 (1, 2, 2, 1),
 (1, 2, 0, 1),
 (2, 1, 1, 2),
 (2, 1, 2, 1),
 (1, 0, 2, 1)]

In [235]:
for (idx1_0, idx1_1, idx2_0, idx2_1) in idxs:
    idx1 = (idx1_0, idx1_1)
    idx2 = (idx2_0, idx2_1)
    v1 = Grid_HMM.two_d_idx_to_vel(idx1)
    v2 = Grid_HMM.two_d_idx_to_vel(idx2)
    b_v1, b_v2 = get_backward_vel(v1, v2)
    b_idx1 = Grid_HMM.two_d_vel_to_idx(b_v1)
    b_idx2 = Grid_HMM.two_d_vel_to_idx(b_v2)
    idx_4d = idx1 + idx2
    b_idx_4d = b_idx1 + b_idx2
    for idx_ in np.ndindex(transition_probs.shape[:2]):
        assert transition_probs[idx_+idx_4d] == transition_probs[idx_+b_idx_4d]

In [236]:
for idx_ in np.ndindex(transition_probs.shape[:2]):
    sum_ = 0
    for idx in idxs:
        sum_ += transition_probs[idx_+idx]
    print(sum_)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.5
0.5
0.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.5
0.5
0.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.5
0.5
0.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.5
0.5
0.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.5
0.5
0.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [181]:
transition_probs.shape[:-2] + (5,)

(32, 32, 3, 3, 5)

In [184]:
trans_probs.ndim

6

In [189]:
trans_probs.size

[autoreload of Net failed: Traceback (most recent call last):
  File "/local/home/ful7rng/.anaconda/envs/ful/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named Net
]


82944

In [237]:
x = np.random.randn(3, 3, 5, 5)

[autoreload of Net failed: Traceback (most recent call last):
  File "/local/home/ful7rng/.anaconda/envs/ful/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named Net
]
[autoreload of propagation.bofum failed: Traceback (most recent call last):
  File "/local/home/ful7rng/.anaconda/envs/ful/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
NameError: name 'config' is not defined
]


In [245]:
x[:, :, *(4, 4)].shape

SyntaxError: invalid syntax (<ipython-input-245-70faed3677bb>, line 1)

In [249]:
x[:, :, *[4, 4]].shape

SyntaxError: invalid syntax (<ipython-input-249-817642c211bb>, line 1)

In [246]:
np.index

[autoreload of propagation.bofum failed: Traceback (most recent call last):
  File "/local/home/ful7rng/.anaconda/envs/ful/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
NameError: name 'config' is not defined
]


AttributeError: 'module' object has no attribute 'index'

In [36]:
type(config)

module

In [38]:
isinstance(config, )

TypeError: isinstance() arg 2 must be a class, type, or tuple of classes and types

### check trajectory on map

In [91]:
import imp
config_path = '/local/home/ful7rng/projects/transition/config_diagonal_True_conditional_True.py'
cf = imp.load_source('config', config_path)

In [92]:
import random
map_name = random.sample(cf.training_maps, 1)[0]
print(map_name)

simple_map


In [93]:
map_path = cf.map_folder + '/' + map_name + '/thresholded_20.png'
trajectory_path = cf.data_folder+'/trajectories/'+map_name+'/'+cf.algo_str+'/'+cf.algo_str+'.npy'
probs_path = cf.data_folder+'/transition_maps/'+map_name+'/'+cf.algo_str+'/'+cf.algo_str+'.npy'
print(map_path)
print(trajectory_path)

/local/home/ful7rng/projects/transition/data/maps/simple_map/thresholded_20.png
/local/home/ful7rng/projects/transition/data_diagonal_true_conditioanl_true/trajectories/simple_map/astar_cost_diagonal_transverse_0_10_5/astar_cost_diagonal_transverse_0_10_5.npy


In [94]:
from utils.occ_map_utils import load_map, show_map
map_, _, _ = load_map(map_path)
trajs = np.load(trajectory_path)
rescale = lambda traj: np.round((traj / .2) - 0.5).astype(int)
trajs_rescaled = map(rescale, trajs)
probs = np.load(probs_path)
print("total %d trajs" % len(trajs))

total 4000 trajs


In [99]:
fig = plt.figure(figsize=(8, 8))
map_axes = fig.add_subplot(111)
plot = Plot(map_axes, map_, .2)
fig_1 = plt.figure(figsize=(5, 5))
fig.canvas.mpl_connect('button_press_event', lambda event: onclick_traj(event, fig, plot, fig_1, trajs_rescaled, probs))

7

[28 46]
[28, 46]
Found 0 trajs going through here
[21 45]
[21, 45]
Found 116 trajs going through here


In [113]:
from matplotlib import cm
from matplotlib.lines import Line2D
def onclick_traj(event,fig, plot, fig_1, raw_trajs, probs, **kwargs):
    
    try:
        for i in range(len(plot.lines)):
            plot.axes.lines.remove(plot.lines[i])
    except AttributeError, ValueError:
        pass

    ix, iy = event.xdata, event.ydata
    coords = np.floor(np.array([ix, iy]) / 0.2).astype(int)
    print(coords)
    
    clicked = np.zeros_like(map_)
    x, y = coords[0], coords[1]
    clicked[x, y] = 1
    plot.set_axes_data("occupancy_axes", clicked)
    
    w, h = kwargs.get('w', 0)//2, kwargs.get('h', 0)//2
    x_padded, y_padded = x+w, y+h
    print([x_padded, y_padded])
    trajs_here = []
    for idx, traj in enumerate(raw_trajs):
        if [x_padded, y_padded] in traj.tolist():
            traj_ = raw_trajs[idx] - np.array([w, h])
            trajs_here.append(rescale_trajectory(traj_, .2))
    print("Found %d trajs going through here" % len(trajs_here))
    colors = cm.Dark2(np.linspace(0, 1, len(trajs_here)))
    plot.lines = map(lambda color: plot.axes.add_line(Line2D([], [], zorder=13, color=color)), colors)
    for idx, line in enumerate(plot.lines):
        xs, ys = trajs_here[idx].T[0], trajs_here[idx].T[1]
#         if idx <= 5:
#             print(trajs_here[idx])
        line.set_data(xs, ys)
    
    fig.canvas.draw()
    
    fig_1.clear()
    plot_4d_tensor(probs[x, y], fig=fig_1)
    fig_1.canvas.draw()

In [304]:
from matplotlib import cm
from matplotlib.lines import Line2D
from utils import nd_gaussian
def onclick_traj_random(event,fig, plot, fig_1, raw_trajs, probs, **kwargs):
    
    try:
        for i in range(len(plot.lines)):
            plot.axes.lines.remove(plot.lines[i])
    except AttributeError, ValueError:
        pass

    ix, iy = event.xdata, event.ydata
    coords = np.floor(np.array([ix, iy]) / 0.2).astype(int)
    print(coords)
    
    clicked = np.zeros_like(map_)
    x, y = coords[0], coords[1]
    clicked[x, y] = 1
    plot.set_axes_data("occupancy_axes", clicked)
    
    g_len = 300
    gaussian_res = float(g_len) / 32
    center = [round((x+.5) * gaussian_res), round((y+.5) * gaussian_res)]
    gaussian = nd_gaussian([g_len, g_len], center, cov=14)
    plot.set_axes_data("gaussian_axes", gaussian)
    
    w, h = kwargs.get('w', 0)//2, kwargs.get('h', 0)//2
    x_padded, y_padded = x+w, y+h
    print([x_padded, y_padded])
    trajs_here = []
    for idx, traj in enumerate(raw_trajs):
        if [x_padded, y_padded] in traj.tolist():
            traj_ = raw_trajs[idx] - np.array([w, h])
            trajs_here.append(rescale_trajectory(traj_, .2))
    print("Found %d trajs going through here" % len(trajs_here))
    idx = np.random.randint(len(trajs_here))
    print("Take random traj at idx %d" % idx)
    idx = 3
    colors = cm.Dark2(np.linspace(0, 1, len(trajs_here)))
    plot.lines = [plot.axes.add_line(Line2D([], [], zorder=13, color=colors[idx]))]

    xs, ys = trajs_here[idx].T[0], trajs_here[idx].T[1]
#         if idx <= 5:
#             print(trajs_here[idx])
    plot.lines[0].set_data(xs, ys)
    
    fig.canvas.draw()
    
    fig_1.clear()
    plot_4d_tensor(probs[x, y], fig=fig_1)
    fig_1.canvas.draw()

In [95]:
trajs[0][:5]

array([[ 3, 48],
       [ 4, 48],
       [ 5, 47],
       [ 5, 46],
       [ 5, 45]])

# new data generator

### traj generator

In [220]:
fname = '/home/ful7rng/projects/transition/data/selected_scenes_from_100_11.npy'
sample_rate = 3
scenes = get_scenes(random_file=False,
                  min_time_interval=3.5,
                  max_time_interval=1e6,
                  sample_rate=sample_rate,
                  file_name=fname)[0]
idx = np.random.choice(np.arange(len(scenes)))
print(idx)
scene = scenes[idx]
map_ = scene.static_map
print(scene.hits.shape)

load file: /home/ful7rng/projects/transition/data/selected_scenes_from_100_11.npy
Found 1655 scenes in total
42
12000000
Found 1160.000000 scenes (= 70.09%) whose interval is in range (3.50, 1000000.00)
Found 0 scenes (= 0.000%) whose interval does not match number of hits.
514
(17, 32, 32)


In [311]:
show_map(scenes[514].static_map)

In [221]:
animate_scenes([scene])

In [79]:
def make_T_section(size):
    map_size = size
    map_ = np.ones((map_size, map_size), dtype=bool)
    lane_length = size // 2

    for j in range(map_size / 2):
        map_[j + 1:j + lane_length + 1, j] = 0
    for j in range(map_size / 2, map_size):
        map_[map_size - j:map_size - j + lane_length, j] = 0
    for j in range(map_size / 2 - lane_length / 2, map_size / 2 + lane_length / 2 + 1):
        map_[map_size / 2:map_size, j] = 0
    x, y = map_size - map_size / 4, map_size / 2
    return map_, x, y

In [139]:
map_, x, y = make_T_section(91)

In [223]:
w, h = map_.shape
padded_map = np.pad(map_, ((w//2, w//2), (h//2, h//2)), mode='edge')

In [213]:
import random

def get_cost_map(map_):
    collision_cost = 2000

    blur1_width = 5
    blurred_map1 = cv2.GaussianBlur(map_.astype(float),
                                   (blur1_width, blur1_width), 0)
    blur2_width = 11
    blurred_map2 = cv2.GaussianBlur(map_.astype(float),
                                   (blur2_width, blur2_width), 0)
    blur1_cost = 2
    blur2_cost = 1
    cost_map = collision_cost * map_.astype(float) + \
               blur1_cost * blurred_map1 + blur2_cost * blurred_map2
    return cost_map

def sample_trajs(padded_map, map_, w, h, num_trajectories, diagonal=False, mode='random', verbose=True):
    
    trajectories = []
    
    cost_map = get_cost_map(padded_map)
    
    start_locations = np.array(np.where(np.logical_not(map_))).T
    start_locations += np.array([w//2, h//2])

    if mode == 'traverse':
        num_locations = len(start_locations)
        for idx, start in enumerate(start_locations):
            if verbose and idx % 1 == 0:
                print('sampling trajectories for possible start locations: {}/{}'.format(
                    idx + 1, num_locations))
                x, y = start
#                 center_map = np.ones_like(padded_map)
#                 center_map[x-w//2:x+w//2, y-h//2:y+h//2] = padded_map[x-w//2:x+w//2, y-h//2:y+h//2]
                center_map = padded_map
                trajectories_ = _sample_trajectories(start, center_map, cost_map,
                                                     num_trajectories, diagonal)
                if trajectories_ is not None:
                    trajectories += trajectories_
    elif mode == 'random':
        for ix in range(int(num_trajectories)):
            if verbose and ix % 10 == 0:
                print('sampling trajectories: {}/{}'.format(
                    ix + 1, num_trajectories))
            # loop to find valid path
            while True:
                start = random.sample(start_locations, 1)[0]
                x, y = start
                center_map = padded_map
#                 center_map = np.ones_like(padded_map)
#                 center_map[x-w//2:x+w//2, y-h//2:y+h//2] = padded_map[x-w//2:x+w//2, y-h//2:y+h//2]
                print("start is :{}".format(start))
                trajectory = _sample_trajectories(start, center_map, cost_map, 1, diagonal)
                if trajectory is not None and len(trajectory) > 0:
                    break
            trajectories += trajectory

    return trajectories

In [214]:

def sample_sg_random(goal_arr):
    # get list of pixels in that area
    goals = np.array(np.where(free_space(goal_arr))).T.tolist()
    # sample random pixel
    if goals:
        goal = random.sample(goals, 1)
        return np.array((goal[0]))
    # in case there is no possible sample result
    else:
        return None

import cv2

from data_generator.multi_agent_planning.scripts.pyastar import astar_path
def _sample_trajectories(start, map_arr, cost_map, num, diagonal=True):
    """ Try to sample num trajectories given a valid start and cost map. """

    # a_star_grid = AStarGrid(cost_map, diagonal=diagonal)

    trajectories = []
    # get wall locations
    walls = set([tuple(loc) for loc in np.array(np.where(map_arr)).T.tolist()])

    ends = []
    current_have = 0
    tried = 0

    while True:

        if current_have >= num:
            break

        if tried >= 10 * num:
            break

        end = sample_sg_random(map_arr)

        if end is None:
            # return None to indicate there is no possible end
            # therefore no possible trajectory for this start
            return None


        if list(end) in ends:
            tried += 1
            continue

        #t1 = time.time()

        trajectory, values = astar_path(cost_map.astype('float32'), start, end, 1, diagonal)
        
        if len(trajectory) == 0:
            trajectory = None
        else:
            trajectory = np.insert(trajectory, 0, end, axis=0)[::-1]
        

        if trajectory is not None:
            # check whether trajectory goes through walls
            traj_traces = [tuple(loc) for loc in trajectory]
            if len(walls.intersection(traj_traces)) > 0:
                trajectory = None



        if trajectory is not None:
            trajectories.append(np.array(trajectory))
            ends.append(list(end))
            current_have += 1
                # print("got one traj")

        tried += 1

    return trajectories

def rescale_trajectory(trajectory, resolution):
    # scale path to real world extent
    return np.array(trajectory + np.array([0.5] * 2)[None, :]) * resolution

In [258]:
from matplotlib import cm
from matplotlib.lines import Line2D
def onclick_generate_traj(event, fig, plot):
    print("clicked")
    
    try:
        for i in range(len(plot.lines)):
            plot.axes.lines.remove(plot.lines[i])
    except AttributeError, ValueError:
        pass

    trajs_rescaled = []
    trajs = sample_trajs(padded_map, map_, w, h, 5, diagonal=True, mode='random')
    for traj in trajs:
        trajs_rescaled.append(rescale_trajectory(traj, .2))
        

    colors = cm.Dark2(np.linspace(0, 1, len(trajs)))
    plot.lines = map(lambda color: plot.axes.add_line(Line2D([], [], zorder=13, color=color)), colors)
    for idx, line in enumerate(plot.lines):
        xs, ys = trajs_rescaled[idx].T[0], trajs_rescaled[idx].T[1]
#         if idx <= 5:
#             print(trajs_here[idx])
        line.set_data(xs, ys)
    
    fig.canvas.draw()
    

In [259]:
fig_1 = plt.figure(figsize=(7, 7))
ax_1 = fig_1.add_subplot(111)
plot_1 = Plot(ax_1, padded_map, .2)
fig_1.canvas.mpl_connect('button_press_event', lambda event: onclick_generate_traj(event, fig_1, plot_1))

7

clicked
sampling trajectories: 1/5
start is :[42 29]
start is :[27 33]
start is :[38 45]
start is :[36 29]
start is :[32 38]
start is :[16 36]
clicked
sampling trajectories: 1/5
start is :[46 33]
start is :[47 35]
start is :[45 19]
start is :[39 43]
start is :[35 22]


### add trajs to transitional map

In [225]:
raw_trajs = sample_trajs(padded_map, map_, w, h, 4, diagonal=True, mode='traverse', verbose=True)

sampling trajectories for possible start locations: 1/615
sampling trajectories for possible start locations: 2/615
sampling trajectories for possible start locations: 3/615
sampling trajectories for possible start locations: 4/615
sampling trajectories for possible start locations: 5/615
sampling trajectories for possible start locations: 6/615
sampling trajectories for possible start locations: 7/615
sampling trajectories for possible start locations: 8/615
sampling trajectories for possible start locations: 9/615
sampling trajectories for possible start locations: 10/615
sampling trajectories for possible start locations: 11/615
sampling trajectories for possible start locations: 12/615
sampling trajectories for possible start locations: 13/615
sampling trajectories for possible start locations: 14/615
sampling trajectories for possible start locations: 15/615
sampling trajectories for possible start locations: 16/615
sampling trajectories for possible start locations: 17/615
sampli

sampling trajectories for possible start locations: 140/615
sampling trajectories for possible start locations: 141/615
sampling trajectories for possible start locations: 142/615
sampling trajectories for possible start locations: 143/615
sampling trajectories for possible start locations: 144/615
sampling trajectories for possible start locations: 145/615
sampling trajectories for possible start locations: 146/615
sampling trajectories for possible start locations: 147/615
sampling trajectories for possible start locations: 148/615
sampling trajectories for possible start locations: 149/615
sampling trajectories for possible start locations: 150/615
sampling trajectories for possible start locations: 151/615
sampling trajectories for possible start locations: 152/615
sampling trajectories for possible start locations: 153/615
sampling trajectories for possible start locations: 154/615
sampling trajectories for possible start locations: 155/615
sampling trajectories for possible start

sampling trajectories for possible start locations: 280/615
sampling trajectories for possible start locations: 281/615
sampling trajectories for possible start locations: 282/615
sampling trajectories for possible start locations: 283/615
sampling trajectories for possible start locations: 284/615
sampling trajectories for possible start locations: 285/615
sampling trajectories for possible start locations: 286/615
sampling trajectories for possible start locations: 287/615
sampling trajectories for possible start locations: 288/615
sampling trajectories for possible start locations: 289/615
sampling trajectories for possible start locations: 290/615
sampling trajectories for possible start locations: 291/615
sampling trajectories for possible start locations: 292/615
sampling trajectories for possible start locations: 293/615
sampling trajectories for possible start locations: 294/615
sampling trajectories for possible start locations: 295/615
sampling trajectories for possible start

sampling trajectories for possible start locations: 417/615
sampling trajectories for possible start locations: 418/615
sampling trajectories for possible start locations: 419/615
sampling trajectories for possible start locations: 420/615
sampling trajectories for possible start locations: 421/615
sampling trajectories for possible start locations: 422/615
sampling trajectories for possible start locations: 423/615
sampling trajectories for possible start locations: 424/615
sampling trajectories for possible start locations: 425/615
sampling trajectories for possible start locations: 426/615
sampling trajectories for possible start locations: 427/615
sampling trajectories for possible start locations: 428/615
sampling trajectories for possible start locations: 429/615
sampling trajectories for possible start locations: 430/615
sampling trajectories for possible start locations: 431/615
sampling trajectories for possible start locations: 432/615
sampling trajectories for possible start

sampling trajectories for possible start locations: 555/615
sampling trajectories for possible start locations: 556/615
sampling trajectories for possible start locations: 557/615
sampling trajectories for possible start locations: 558/615
sampling trajectories for possible start locations: 559/615
sampling trajectories for possible start locations: 560/615
sampling trajectories for possible start locations: 561/615
sampling trajectories for possible start locations: 562/615
sampling trajectories for possible start locations: 563/615
sampling trajectories for possible start locations: 564/615
sampling trajectories for possible start locations: 565/615
sampling trajectories for possible start locations: 566/615
sampling trajectories for possible start locations: 567/615
sampling trajectories for possible start locations: 568/615
sampling trajectories for possible start locations: 569/615
sampling trajectories for possible start locations: 570/615
sampling trajectories for possible start

In [226]:
trajs = map(lambda traj: rescale_trajectory(traj, .2), raw_trajs)

In [227]:
from data_generator.human_mcm import Grid_HMM
def get_transition_map(trajs, padded_map, w, h, diagonal=False, conditional=True):

    mcm = Grid_HMM(np.array(padded_map.shape).astype(int))
    for idx, trajectory in enumerate(trajs):
        if idx % 100 == 0:
            print("Add transitions of {}/{} trajectory to the mcm.".format(idx+1, len(trajs)))
        for t in range(trajectory.shape[0] - 2):
            
            from_ = trajectory[t, :]
            current = trajectory[t + 1, :]
            to = trajectory[t + 2, :]
            mcm.add_transition(from_, current, to)

    transition_probs = mcm.get_transition_probs(conditional, diagonal)
    return transition_probs[w//2:w+w//2, h//2:h+h//2]

In [228]:
transition_probs = get_transition_map(raw_trajs, padded_map, w, h, diagonal=True)

Add transitions of 1/2385 trajectory to the mcm.
Add transitions of 101/2385 trajectory to the mcm.
Add transitions of 201/2385 trajectory to the mcm.
Add transitions of 301/2385 trajectory to the mcm.
Add transitions of 401/2385 trajectory to the mcm.
Add transitions of 501/2385 trajectory to the mcm.
Add transitions of 601/2385 trajectory to the mcm.
Add transitions of 701/2385 trajectory to the mcm.
Add transitions of 801/2385 trajectory to the mcm.
Add transitions of 901/2385 trajectory to the mcm.
Add transitions of 1001/2385 trajectory to the mcm.
Add transitions of 1101/2385 trajectory to the mcm.
Add transitions of 1201/2385 trajectory to the mcm.
Add transitions of 1301/2385 trajectory to the mcm.
Add transitions of 1401/2385 trajectory to the mcm.
Add transitions of 1501/2385 trajectory to the mcm.
Add transitions of 1601/2385 trajectory to the mcm.
Add transitions of 1701/2385 trajectory to the mcm.
Add transitions of 1801/2385 trajectory to the mcm.
Add transitions of 1901/

In [229]:
transition_probs.shape

(32, 32, 3, 3, 3, 3)

In [182]:
diagonal=True
if diagonal:
    velocities = [[0, 1], [1, 0], [0, -1], [-1, 0],
                  [1, 1], [1, -1], [-1, 1], [-1, -1]]
else:
    velocities = [[0, 1], [1, 0], [0, -1], [-1, 0]]

idxs = map(lambda vel: Grid_HMM.two_d_vel_to_idx(vel), velocities)
print(idxs)

[(1, 2), (2, 1), (1, 0), (0, 1), (2, 2), (2, 0), (0, 2), (0, 0)]


In [230]:
count = 0
# keep motion for empty entries
for idx_xy in np.ndindex(map_.shape):
    for idx_vel_last in idxs:
        if np.all(transition_probs[idx_xy+idx_vel_last]==0):
            transition_probs[idx_xy+idx_vel_last+idx_vel_last] = 1
            count += 1
print("fill in %d entries to keep motion" % count)

fill in 0 entries to keep motion


In [126]:
x, y = np.random.randint(32), np.random.randint(32)
vel_x, vel_y = np.random.randint(3), np.random.randint(3)
print(x, y, vel_x, vel_y)
transition_probs[x, y, vel_x, vel_y]

(4, 24, 0, 2)


array([[ 0.,  0.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [231]:
def isclose(a, b, rel_tol=1e-09, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

In [232]:
for idx in np.ndindex(map_.shape):
    if not isclose(transition_probs[idx].sum(), 8):
        print(idx)
        print(transition_probs[idx].sum())

In [306]:

fig = plt.figure(figsize=(8, 8))
map_axes = fig.add_subplot(111)
plot = Plot(map_axes, map_, .2, title='')
from utils.occ_map_utils import greens_cm
plot.add_custom_image("gaussian_axes", image=np.zeros((300, 300)), cmap=greens_cm, extent=(0, 6.4, 0. ,6.4), zorder=-1)
plot.axes.set_xlabel('m')
plot.axes.set_ylabel('m')
fig_1 = plt.figure(figsize=(5, 5))
fig.canvas.mpl_connect('button_press_event', lambda event: onclick_traj_random(event, fig, plot, fig_1, raw_trajs, transition_probs, w=w, h=h))

7

In [187]:
np.save("/home/ful7rng/Desktop/thesis/writting_materials/91_T_section_probs_cost_1_2_cost_2_1.npy", transition_probs)

### blur spatially

In [188]:
blurred_probs = transition_probs.copy()

In [189]:
turns = []
for idx_xy in np.ndindex(map_.shape):
    for idx_vel_last in idxs:
        max_idx = np.argmax(transition_probs[idx_xy+idx_vel_last])
        max_idx = np.unravel_index(max_idx, (3, 3))
        if not max_idx == idx_vel_last:
            turns.append(idx_xy+idx_vel_last)

In [190]:
len(turns)

3240

In [133]:
plt.figure()
ax = plt.imshow(transition_probs[turns[0]])
print(turns[0])
plt.colorbar(ax)

(4, 0, 0, 1)


In [154]:
from utils import nd_gaussian
blur_extent = 9
var = 1.2
half_be = blur_extent // 2
gaussian = nd_gaussian((blur_extent, blur_extent), (half_be, half_be), var)
gaussian /= gaussian.max()
plt.figure()
ax = plt.imshow(gaussian)
plt.colorbar(ax)

In [191]:
blurred_probs = transition_probs.copy()
blurred_probs = np.pad(blurred_probs, ((half_be, half_be), (half_be, half_be),
                           (0, 0), (0, 0), (0, 0), (0, 0)),
                            mode='constant', constant_values=0)

for turn in turns:
    temp = gaussian[..., None, None] * transition_probs[turn]
    x, y = turn[0], turn[1]
    vel_x, vel_y = turn[2], turn[3]
    blurred_probs[x:x+blur_extent, y:y+blur_extent, vel_x, vel_y] += temp
blurred_probs = blurred_probs[half_be:half_be + w, half_be:half_be + h]

# normalize
with np.errstate(divide='ignore', invalid='ignore'):
    blurred_probs /= blurred_probs.sum(axis=(4, 5), keepdims=True)
    blurred_probs[~np.isfinite(blurred_probs)] = 0


In [192]:
for idx in np.ndindex(map_.shape):
    if not isclose(blurred_probs[idx].sum(), 8):
        print(idx)
        print(blurred_probs[idx].sum())

In [83]:
blurred_probs.shape

(32, 32, 3, 3, 3, 3)

In [411]:
blurred_probs[23, 21, 1, 2]

array([[  0.00000000e+00,   0.00000000e+00,   4.74501979e-10],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [193]:
np.save("/home/ful7rng/Desktop/thesis/writting_materials/91_T_section_probs_blurred_var_1.2_extent_9_cost_1_2_cost_2_1.npy", blurred_probs)

In [307]:
map_trajs_example = {}
map_trajs_example['map'] = map_
map_trajs_example['trajectories'] = raw_trajs
map_trajs_example['probs'] = transition_probs
map_trajs_example['blurred_probs_var_1.2_extent_9'] = blurred_probs

In [309]:
from utils import pickle_save

In [310]:
pickle_save("/home/ful7rng/Desktop/thesis/writting_materials/map_traj_example.pkl", map_trajs_example)

### tracking

In [158]:
from utils.plot_utils import plot_4d_tensor

def onclick(event, models, anim, fig_1, fig_2, fig_3, ma_plot, vel_plot, merge_vel_plot, final_vel_plot):
    ix, iy = event.xdata, event.ydata
    coords = np.floor(np.array([ix, iy]) / models[0].map_res).astype(int)
    print(coords)
    
    all_axes = [plot.axes for plot in anim.plots]
    
    for i, ax in enumerate(all_axes):

        # For infomation, print which axes the click was in
        if ax == event.inaxes:
            print "Click is in axes ax{}".format(i+1)
            break
            
    clicked = np.zeros_like(models[0].map)
    x, y = coords[0], coords[1]
    clicked[x, y] = 1
    for plot in anim.plots:
        plot.set_axes_data("occupancy_axes", clicked)
    anim.fig.canvas.draw()
    
    fig_1.clear()
    fig_2.clear()
    
    
    if models[i].kernels.ndim == 6:
        kernel = models[i].kernels[x, y]
        condi_prob = models[i].nn_probs[x, y]
    else:
        kernel = models[i].kernels
        condi_prob = models[i].kernels
        
#     print(kernel.shape)
#     print(condi_prob.shape)
    print(np.sum(kernel, axis=(2, 3)))
    plot_4d_tensor(condi_prob, fig=fig_1)
    plot_4d_tensor(kernel, fig=fig_2)
    if hasattr(models[i], 'apply_vel_mask') and models[i].apply_vel_mask:
        print("plot vel mask")
        ma_plot.set_axes_data("occupancy_axes", models[i].vel_mask[x, y])
    else:
        ma_plot.set_axes_data("occupancy_axes", models[i].ma_vel)
    vel_plot.set_axes_data("occupancy_axes", models[i].P_Vt_pred[x, y])
    merge_vel_plot.set_axes_data("occupancy_axes", models[i].P_Vt_merged[x, y])
    final_vel_plot.set_axes_data("occupancy_axes", models[i].P_Vt[x, y])
    #print(models[i].P_Vt[x, y])
    fig_1.canvas.draw()
    fig_2.canvas.draw()
    fig_3.canvas.draw()
    
    model_names = map(lambda model: model.name, models)
    occs = map(lambda model: model.P_Ot[x, y], models)
    for name, occ in zip(model_names, occs):
        print("loc ({}, {}) of model {} has occupancy of {}".format(x, y, name, occ))

In [159]:
def on_press(event, animation):
    global scenes
    if event.key == 'n':
        idx = np.random.choice(np.arange(len(scenes)))
        print(idx)
        scene = scenes[idx]
        animation.update(scene)

In [160]:
diagonal = True
if diagonal:
    model_name = '23_ALL_MAPS_EIGHT_DIRECTIONS'
    model_name = '04_DIAGONAL_TRUE_CONDITIONAL_TRUE'
else:
    # model_name = '20_ALL_MAPS_W_MASK'
    model_name = '04_DIAGONAL_FALSE_CONDITIONAL_FALSE'

In [197]:
force_predict = True
simulated_data = False
kwargs = dict(omega=0.01,
                  extent=7,
                  noise_var=0.3,
                  measurement_lost=8,
                  verbose=False)

naive_bofum = naiveBOFUM(map_, name='BOFUM', simulated_data=simulated_data, **kwargs)
bofum = conditionalBOFUM(map_, model_name,
                               name = 'BOFUM acc. with new trans_porbs',
                               simulated_data=simulated_data,
                               force_predict=force_predict,
                               acceleration_interpretation=True,
                               nn_probs = transition_probs,
                               **kwargs)

# bofum_old = conditionalBOFUM(map_, model_name,
#                                name = 'BOFUM with n. o. as acceleration',
#                                simulated_data=simulated_data,
#                                force_predict=force_predict,
#                                acceleration_interpretation=True,
#                                **kwargs)

# bofum_blur = conditionalBOFUM(map_, model_name,
#                                name = 'BOFUM acc new with blur_spatially',
#                                simulated_data=simulated_data,
#                                force_predict=force_predict,
#                                acceleration_interpretation=True,
#                                nn_probs = blurred_probs,
#                                **kwargs)

# bofum_blur_ma = conditionalBOFUM(map_, model_name,
#                                name = 'BOFUM acc new with blur and ma',
#                                simulated_data=simulated_data,
#                                force_predict=force_predict,
#                                acceleration_interpretation=True,
#                                keep_motion=True,
#                                nn_probs = blurred_probs,
#                                **kwargs)

In [198]:
models = []
models.append(naive_bofum)
models.append(bofum)
# models.append(bofum_blur)
# models.append(bofum_blur_ma)
# models.append(bofum_old)

# models.append(bofum_ma)
# models.append(bofum_vm)
# models.append(bofum_keep)
# models.append(bofum_vm_blur)

In [199]:
scene_copy = deepcopy(scene)
processed_scene_copy = BOFUMRealdata.scene_preprocessing(scene_copy)
#animate_scenes([processed_scene_copy])
laser_frequency = 12
tracking_to = processed_scene_copy.hits.shape[0]
num_steps = tracking_to

In [200]:
scene_generator = naiveBOFUM(map_, name='BOFUM', simulated_data=True, noise_var=0.6)

In [201]:
num_steps = 48
scene_generator.initialize(np.random.randint(1, 4), num_steps, diagonal=True)
scene = scene_generator.trajs_to_scene()

sampling trajectories: 1/1
time out
sampling trajectories: 1/1
time out
sampling trajectories: 1/1
time out
sampling trajectories: 1/1


In [202]:

ani = TrackingAnimRealdata(models, num_steps, scene,
                                plot_map=True,
                                plot_seen=True)

fig_1 = plt.figure(figsize=(6, 5))
fig_2 = plt.figure(figsize=(6, 5))
fig_size = (5 * 4, 5)
fig_3 = plt.figure(figsize=fig_size)
ma_ax = fig_3.add_subplot(141)
ma_plot = Plot(ma_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='moving average vel/ vel mask')
vel_ax = fig_3.add_subplot(142)
vel_plot = Plot(vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='predicted vel')
merge_vel_ax = fig_3.add_subplot(143)
merge_vel_plot = Plot(merge_vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='merged velocity')
final_vel_ax = fig_3.add_subplot(144)
final_vel_plot = Plot(final_vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='velocity')

ani.fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, models, ani, fig_1, fig_2, fig_3, ma_plot, vel_plot, merge_vel_plot, final_vel_plot))
ani.fig.canvas.mpl_connect('key_press_event', lambda event: on_press(event, ani))

frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7


12

[40 57]
Click is in axes ax2
[[ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]]
loc (40, 57) of model BOFUM has occupancy of 3.02788097625e-17
loc (40, 57) of model BOFUM acc. with new trans_porbs has occupancy of 3.36431219583e-17
[42 67]
Click is in axes ax2
[[ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]]
loc (42, 67) of model BOFUM has occupancy of 3.25216845597e-17
loc (42, 67) of model BOFUM acc. with new trans_porbs has occupancy of 5.27075577347e-17
frame = 8
frame = 9
frame = 10
frame = 11
frame = 12
frame = 13
frame = 14
frame = 15
frame = 16
frame = 17
frame = 18
frame = 19
frame = 20
frame = 21
frame = 22
frame = 23
frame = 24
frame = 25
frame 

In [88]:
np.sum(blurred_probs[28, 20], axis=(2, 3))

array([[ 1.,  1.,  1.],
       [ 1.,  0.,  1.],
       [ 1.,  1.,  1.]])

frame = 7
motion factor :0.5
frame = 8
motion factor :0.3
frame = 9
motion factor :0.18
frame = 10
motion factor :0.108
frame = 11
motion factor :0.0648
frame = 12
motion factor :0.03888
frame = 13
motion factor :0.023328
frame = 14
motion factor :0.0139968
frame = 15
motion factor :0.00839808
frame = 16
motion factor :0.005038848
frame = 17
motion factor :0.0030233088
frame = 18
motion factor :0.00181398528
frame = 19
motion factor :0.001088391168
frame = 20
motion factor :0.0006530347008
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
motion factor :0.5
frame = 8
motion factor :0.3
frame = 9
motion factor :0.18
frame = 10
motion factor :0.108
frame = 11
motion factor :0.0648
frame = 12
motion factor :0.03888
[18 17]
Click is in axes ax4
[[ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]]
l